# Foxglove Data Platform Demo Notebook

This notebook demonstrates retrieving data and events from Foxglove Data Platform.


In [ ]:
# First let's we can create an instance of our client.

from datetime import datetime

import pandas as pd
from foxglove_data_platform.client import Client

client = Client(token="fox_sk_CFlhbEgxMQbHBQGRCNE7c3CBaOxtrpk5")

In [ ]:
# Next let's use the client to find our data coverage.

coverage = client.get_coverage(start=datetime(2018, 1, 1), end=datetime(2019, 1, 1))
coverage = sorted(coverage, key=lambda c: c["start"])
pd.DataFrame(coverage).head()

In [ ]:
# Now let's download some GPS messages.

gps_messages = [
    (message.latitude, message.longitude)
    for topic, record, message in client.get_messages(
        device_id=coverage[1]["device_id"],
        start=coverage[1]["start"],
        end=coverage[1]["end"],
        topics=["/gps"],
    )
]
pd.DataFrame(gps_messages, columns=["lat", "lon"]).head()

In [ ]:
# We can use our GPS messages to plot our course on the map.

import folium

figure = folium.Figure(width=640, height=480)
map = folium.Map(location=gps_messages[0], zoom_start=200, width="100%")
folium.PolyLine(
    locations=gps_messages,
    weight=10,
    color="purple",
).add_to(map)
map.add_to(figure)

In [ ]:
# We can fetch acceleration messages that span multiple imports

imu_messages = [
    {
        "time": pd.Timestamp(message.header.stamp.to_nsec(), unit="ns").isoformat(),
        "accel_x": message.linear_acceleration.x,
        "accel_y": message.linear_acceleration.y,
    }
    for topic, record, message in client.get_messages(
        device_id=coverage[0]["device_id"],
        start=coverage[0]["start"],
        end=coverage[-1]["end"],
        topics=["/imu"],
    )
]
pd.DataFrame(imu_messages).plot(x="time", figsize=(10, 6), rot=45)

In [ ]:
marker_messages = client.get_messages(
    device_id=coverage[1]["device_id"],
    start=coverage[1]["start"],
    end=coverage[1]["end"],
    topics=["/markers/annotations"],
)

In [ ]:
import matplotlib as mpl
from pandasql import sqldf

color_to_classname = {
    "#000000": "noise",
    "#468250": "animal",
    "#0000e6": "human.pedestrian.adult",
    "#87ceeb": "human.pedestrian.child",
    "#f08080": "human.pedestrian.construction_worker",
    "#db7093": "human.pedestrian.personal_mobility",
    "#000080": "human.pedestrian.police_officer",
    "#f08080": "human.pedestrian.stroller",
    "#8a2be2": "human.pedestrian.wheelchair",
    "#708090": "movable_object.barrier",
    "#d2691e": "movable_object.debris",
    "#696969": "movable_object.pushable_pullable",
    "#2f4f4f": "movable_object.trafficcone",
    "#bc8f8f": "static_object.bicycle_rack",
    "#dc143c": "vehicle.bicycle",
    "#ff7f50": "vehicle.bus.bendy",
    "#ff4500": "vehicle.bus.rigid",
    "#ff9e00": "vehicle.car",
    "#e99646": "vehicle.construction",
    "#ffd700": "vehicle.emergency.ambulance",
    "#ffd700": "vehicle.emergency.police",
    "#ff3d63": "vehicle.motorcycle",
    "#ff8c00": "vehicle.trailer",
    "#ff6347": "vehicle.truck",
    "#00cfbf": "flat.driveable_surface",
    "#af004b": "flat.other",
    "#4b004b": "flat.sidewalk",
    "#70b43c": "flat.terrain",
    "#deb887": "static.manmade",
    "#ffe4c4": "static.other",
    "#00af00": "static.vegetation",
    "#fff0f5": "vehicle.ego",
}

flattened_markers = []
for topic, record, message in marker_messages:
    for marker in message.markers:
        color = mpl.colors.to_hex([marker.color.r, marker.color.g, marker.color.b])
        class_name = color_to_classname[color]
        flattened_markers.append((marker.text, class_name))
annotations = pd.DataFrame(flattened_markers, columns=["annotation_id", "class_name"])

pysqldf = lambda q: sqldf(q, globals())
pysqldf(
    "SELECT class_name,COUNT(*) as count FROM annotations GROUP BY class_name ORDER BY count DESC"
)